### Slider Control of `KIMLAB-Dualarms`

In [33]:
import sys
import numpy as np
sys.path.append("../utils")
from mujoco_parser import MuJoCoParserClass
from util import MultiSliderClass,get_idxs
print ("Ready.")

Ready.


#### Load

In [34]:
xml_path = '../asset/kimlab_dualarms/scene_dualarms.xml'
env = MuJoCoParserClass(name='Dual Arms',rel_xml_path=xml_path,VERBOSE=True)
env.set_geom_color(
    rgba                = (0.2,0.2,1.0,0.9),
    body_names_to_color = [
        'robot1/end_link','robot1/index_L1','robot1/index_L2','robot1/index_tip',
        'robot1/middle_L1','robot1/middle_L2','robot1/middle_tip',
        'robot1/ring_L1','robot1/ring_L2','robot1/ring_tip',
        'robot1/pinky_L1','robot1/pinky_L2','robot1/pinky_tip',
        'robot1/thumb_L1','robot1/thumb_L2','robot1/thumb_tip'],
)
env.set_geom_color(
    rgba                = (1.0,0.2,0.2,0.9),
    body_names_to_color = [
        'robot2/end_link','robot2/index_L1','robot2/index_L2','robot2/index_tip',
        'robot2/middle_L1','robot2/middle_L2','robot2/middle_tip',
        'robot2/ring_L1','robot2/ring_L2','robot2/ring_tip',
        'robot2/pinky_L1','robot2/pinky_L2','robot2/pinky_tip',
        'robot2/thumb_L1','robot2/thumb_L2','robot2/thumb_tip'],
)
print ("Done.")

dt:[0.0020] HZ:[500]
n_dof (=nv):[32]
n_geom:[75]
geom_names:[None, None, None, None, None, None, 'floor', None, None, None, None, None, None, None, None, None, None, None, None, 'robot1/wrist_mesh', 'robot1/palm_mesh', 'robot1/index_mesh_1', None, 'robot1/index_mesh_2', None, 'robot1/middle_mesh_1', None, 'robot1/middle_mesh_2', None, 'robot1/ring_mesh_1', None, 'robot1/ring_mesh_2', None, 'robot1/pinky_mesh_1', None, 'robot1/pinky_mesh_2', None, 'robot1/thumb_mesh_1', None, 'robot1/thumb_mesh_2', None, None, None, None, None, None, None, None, None, None, None, None, None, 'robot2/wrist_mesh', 'robot2/palm_mesh', 'robot2/index_mesh_1', None, 'robot2/index_mesh_2', None, 'robot2/middle_mesh_1', None, 'robot2/middle_mesh_2', None, 'robot2/ring_mesh_1', None, 'robot2/ring_mesh_2', None, 'robot2/pinky_mesh_1', None, 'robot2/pinky_mesh_2', None, 'robot2/thumb_mesh_1', None, 'robot2/thumb_mesh_2', None]
n_body:[43]
body_names:['world', 'robot1/link2', 'robot1/link3', 'robot1/link4', 'robot

#### Slider Control

In [35]:
env.reset() # reset
joint_names_to_slide = env.rev_joint_names
idxs_fwd         = env.get_idxs_fwd(joint_names=joint_names_to_slide) # idxs for qpos
idxs_range       = get_idxs(env.joint_names,joint_names_to_slide) # idxs for joint_ranges
q_init           = env.get_qpos_joints(joint_names_to_slide)
n_joint_to_slide = len(joint_names_to_slide)

sliders = MultiSliderClass(
    n_slider      = n_joint_to_slide,
    title         = 'Sliders for Common-Rig Control',
    window_width  = 600,
    window_height = 800,
    x_offset      = 50,
    y_offset      = 100,
    slider_width  = 350,
    label_texts   = joint_names_to_slide,
    slider_mins   = env.joint_ranges[idxs_range,0],
    slider_maxs   = env.joint_ranges[idxs_range,1],
    slider_vals   = q_init,
    resolution    = 0.01,
    VERBOSE       = False,
)

# Initialize MuJoCo viewer
env.init_viewer(
    viewer_title='Common-Rig-H',viewer_width=1200,viewer_height=800,
    viewer_hide_menus=True,
)
env.update_viewer(
    azimuth=177,distance=1.42,elevation=-34,lookat=[0.25,-0.0,2.1],
    VIS_TRANSPARENT=True,
)

# Reset and loop
env.reset()
while env.is_viewer_alive() and sliders.is_window_exists():
    # Update
    sliders.update() # update slider
    qpos_curr = env.data.qpos.copy()
    qpos_curr[idxs_fwd] = sliders.get_slider_values() # get slider values
    coupled_joint_idxs_list = [
        [6,7],[8,9],[10,11],[12,13],
        [22,23],[24,25],[26,27],[28,29]
    ]

    # couple fingers
    q_couple = qpos_curr.copy()
    for i in range(len(coupled_joint_idxs_list)):
        coupled_joint_idxs = coupled_joint_idxs_list[i]
        q_couple[coupled_joint_idxs[1]] = qpos_curr[coupled_joint_idxs[0]] * 1.05851325 + 0.72349796
        
    env.forward(q=q_couple,INCREASE_TICK=True) # FK
    sliders.set_slider_values(slider_values=env.data.qpos[idxs_fwd]) # update slider values

    # for (i,joint_name) in enumerate(env.joint_names):
    #     env.plot_T(p=env.get_p_joint(joint_name),R=np.eye(3),PLOT_AXIS=False,label=joint_name)

    # Render
    if env.loop_every(tick_every=10):
        env.plot_T(p=np.zeros(3),R=np.eye(3,3),
                   PLOT_AXIS=True,axis_len=0.5,axis_width=0.005)
        env.plot_T(p=np.array([0,0,0.5]),R=np.eye(3),PLOT_AXIS=False,
                   label='Tick:[%d]'%(env.tick))
        env.plot_joint_axis(axis_len=0.02,axis_r=0.004,joint_names=joint_names_to_slide) # joint axis
        env.plot_contact_info(h_arrow=0.3,rgba_arrow=[1,0,0,1],PRINT_CONTACT_BODY=True) # contact
        env.render()
        
# Close MuJoCo viewer and sliders
env.close_viewer()
sliders.close()

Pressed ESC
Quitting.
